In [1]:
#import necessary packages
import cv2
import numpy as np
import pandas as pd
import glob
from matplotlib import pyplot as plt

In [2]:
#decalring a class to extract features
class ColorDescriptor:
    def __init__(self, bins):
        # store the number of bins for the 3D histogram
        self.bins = bins

    def describe(self, image):
        # convert the image to the HSV color space and initialize
        # the features used to quantify the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        features = []

    # grab the dimensions and compute the center of the image
        (h, w) = image.shape[:2]
        (cX, cY) = (int(w * 0.5), int(h * 0.5))
        # divide the image into four rectangles/segments (top-left,
        # top-right, bottom-right, bottom-left)
        segments = [(0, cX, 0, cY), (cX, w, 0, cY), (cX, w, cY, h),(0, cX, cY, h)]

        # construct an elliptical mask representing the center of the
        # image
        (axesX, axesY) = (int(w * 0.75) // 2, int(h * 0.75) // 2)
        ellipMask = np.zeros(image.shape[:2], dtype = "uint8")
        cv2.ellipse(ellipMask, (cX, cY), (axesX, axesY), 0, 0, 360, 255, -1)

        # loop over the segments
        for (startX, endX, startY, endY) in segments:
            # construct a mask for each corner of the image, subtracting
            # the elliptical center from it
            cornerMask = np.zeros(image.shape[:2], dtype = "uint8")
            cv2.rectangle(cornerMask, (startX, startY), (endX, endY), 255, -1)
            cornerMask = cv2.subtract(cornerMask, ellipMask)
            # extract a color histogram from the image, then update the
            # feature vector
            hist = self.histogram(image, cornerMask)
            features.extend(hist)

        # extract a color histogram from the elliptical region and
        # update the feature vector
        hist = self.histogram(image, ellipMask)
        features.extend(hist)

        # return the feature vector
        return features

    def histogram(self, image, mask):
        # extract a 3D color histogram from the masked region of the
        # image, using the supplied number of bins per channel
        hist = cv2.calcHist([image], [0, 1, 2], mask, self.bins,[0, 180, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        return hist

In [21]:
# initialize the color descriptor
cd = ColorDescriptor((8, 12, 3))

path = '/home/subarna/Documents/ml_notebooks/assg_1_part2_data'
filenames = glob.glob(path + "/*.jpeg")
dfs = []
for imagePath in filenames:
    imageID = imagePath[imagePath.rfind("/") + 1:]
    image = cv2.imread(imagePath)
     
    # describe the image
    features = cd.describe(image)
 
    # write the features to file
    features = [str(f) for f in features]
    
    #describing a certain column to specify whether th egiven image is either face or night or landscape
    #(labelling of image)
    category = imagePath.split('.')[0]
    if category == '/home/subarna/Documents/ml_notebooks/assg_1_part2_data/face':
        x = 0
    if category == '/home/subarna/Documents/ml_notebooks/assg_1_part2_data/night':
        x = 1
    if category == '/home/subarna/Documents/ml_notebooks/assg_1_part2_data/landscape':
        x = 2
        
    feat = (imageID,x,",".join(features))
    dfs.append(feat)
df = pd.DataFrame(dfs)  #making dataframes
print(df)

                    0  1                                                  2
0        face.12.jpeg  0  0.0044469754,0.051051278,0.0,0.0065815235,0.00...
1        face.22.jpeg  0  0.014324457,0.0,0.8811065,0.0,0.0,0.0,0.0,0.0,...
2   landscape.26.jpeg  2  0.0010765599,0.0010765599,0.0,0.0021531198,0.0...
3   landscape.11.jpeg  2  0.0024333484,0.0,0.0,0.0,0.0,0.0,0.0030416856,...
4    landscape.5.jpeg  2  0.0,0.0,0.0010035756,0.0,0.0,0.0,0.0,0.0,0.0,0...
..                ... ..                                                ...
87      night.27.jpeg  1  0.0040945313,0.0,0.0,0.0002559082,0.0,0.0,0.00...
88  landscape.25.jpeg  2  0.0,0.0,0.0,0.0,0.0,0.0013365867,0.0,0.0,0.002...
89   landscape.9.jpeg  2  0.0,0.0,0.0,0.0,0.0,0.0,0.00067682005,0.0,0.0,...
90  landscape.23.jpeg  2  0.7083379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...
91  landscape.24.jpeg  2  0.00049033243,0.00049033243,0.47660312,0.0,0.0...

[92 rows x 3 columns]


In [22]:
df1 = df[2].str.split(",",expand = True)
df1

0              1             2             3             4     \
0    0.0044469754    0.051051278           0.0  0.0065815235  0.0008893951   
1     0.014324457            0.0     0.8811065           0.0           0.0   
2    0.0010765599   0.0010765599           0.0  0.0021531198  0.0026913998   
3    0.0024333484            0.0           0.0           0.0           0.0   
4             0.0            0.0  0.0010035756           0.0           0.0   
..            ...            ...           ...           ...           ...   
87   0.0040945313            0.0           0.0  0.0002559082           0.0   
88            0.0            0.0           0.0           0.0           0.0   
89            0.0            0.0           0.0           0.0           0.0   
90      0.7083379            0.0           0.0           0.0           0.0   
91  0.00049033243  0.00049033243    0.47660312           0.0  0.0009806649   

            5              6             7             8              9     \
0            0.0      0.0254367           0.0           0.0    0.043046724   
1            0.0            0.0           0.0           0.0            0.0   
2            0.0    0.006459359   0.010227319  0.0069976393   0.0037679595   
3            0.0   0.0030416856           0.0  0.0036500227    0.010950068   
4            0.0            0.0           0.0           0.0            0.0   
..           ...            ...           ...           ...            ...   
87           0.0   0.0002559082           0.0           0.0   0.0012795411   
88  0.0013365867            0.0           0.0  0.0022276444            0.0   
89           0.0  0.00067682005           0.0           0.0  0.00067682005   
90           0.0            0.0           0.0           0.0            0.0   
91           0.0  0.00049033243  0.0063743214  0.0024516622    0.004412992   

    ... 1430           1431 1432 1433           1434 1435 1436           1437  \
0   ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   
1   ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   
2   ...  0.0   0.0010678926  0.0  0.0   0.0008009195  0.0  0.0   0.0024027585   
3   ...  0.0  0.00037591136  0.0  0.0            0.0  0.0  0.0  0.00018795568   
4   ...  0.0  0.00019538656  0.0  0.0  0.00019538656  0.0  0.0            0.0   
..  ...  ...            ...  ...  ...            ...  ...  ...            ...   
87  ...  0.0  0.00062890793  0.0  0.0  0.00015722698  0.0  0.0   0.0022011776   
88  ...  0.0  0.00051748834  0.0  0.0  0.00051748834  0.0  0.0  0.00034499224   
89  ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   
90  ...  0.0            0.0  0.0  0.0  0.00024025064  0.0  0.0  0.00048050127   
91  ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   

   1438 1439  
0   0.0  0.0  
1   0.0  0.0  
2   0.0  0.0  
3   0.0  0.0  
4   0.0  0.0  
..  ...  ...  
87  0.0  0.0  
88  0.0  0.0  
89  0.0  0.0  
90  0.0  0.0  
91  0.0  0.0  

[92 rows x 1440 columns]

In [23]:
del df[2]
for i in range(2,1442):
    df[i] = df1[i-2]
df

0     1              2              3             4     \
0        face.12.jpeg     0   0.0044469754    0.051051278           0.0   
1        face.22.jpeg     0    0.014324457            0.0     0.8811065   
2   landscape.26.jpeg     2   0.0010765599   0.0010765599           0.0   
3   landscape.11.jpeg     2   0.0024333484            0.0           0.0   
4    landscape.5.jpeg     2            0.0            0.0  0.0010035756   
..                ...   ...            ...            ...           ...   
87      night.27.jpeg     1   0.0040945313            0.0           0.0   
88  landscape.25.jpeg     2            0.0            0.0           0.0   
89   landscape.9.jpeg     2            0.0            0.0           0.0   
90  landscape.23.jpeg     2      0.7083379            0.0           0.0   
91  landscape.24.jpeg     2  0.00049033243  0.00049033243    0.47660312   

            5             6             7              8             9     \
0   0.0065815235  0.0008893951           0.0      0.0254367           0.0   
1            0.0           0.0           0.0            0.0           0.0   
2   0.0021531198  0.0026913998           0.0    0.006459359   0.010227319   
3            0.0           0.0           0.0   0.0030416856           0.0   
4            0.0           0.0           0.0            0.0           0.0   
..           ...           ...           ...            ...           ...   
87  0.0002559082           0.0           0.0   0.0002559082           0.0   
88           0.0           0.0  0.0013365867            0.0           0.0   
89           0.0           0.0           0.0  0.00067682005           0.0   
90           0.0           0.0           0.0            0.0           0.0   
91           0.0  0.0009806649           0.0  0.00049033243  0.0063743214   

    ... 1432           1433 1434 1435           1436 1437 1438           1439  \
0   ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   
1   ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   
2   ...  0.0   0.0010678926  0.0  0.0   0.0008009195  0.0  0.0   0.0024027585   
3   ...  0.0  0.00037591136  0.0  0.0            0.0  0.0  0.0  0.00018795568   
4   ...  0.0  0.00019538656  0.0  0.0  0.00019538656  0.0  0.0            0.0   
..  ...  ...            ...  ...  ...            ...  ...  ...            ...   
87  ...  0.0  0.00062890793  0.0  0.0  0.00015722698  0.0  0.0   0.0022011776   
88  ...  0.0  0.00051748834  0.0  0.0  0.00051748834  0.0  0.0  0.00034499224   
89  ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   
90  ...  0.0            0.0  0.0  0.0  0.00024025064  0.0  0.0  0.00048050127   
91  ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   

   1440 1441  
0   0.0  0.0  
1   0.0  0.0  
2   0.0  0.0  
3   0.0  0.0  
4   0.0  0.0  
..  ...  ...  
87  0.0  0.0  
88  0.0  0.0  
89  0.0  0.0  
90  0.0  0.0  
91  0.0  0.0  

[92 rows x 1442 columns]

In [25]:
x_train = df1
x_train #training dataframe of features

0              1             2             3             4     \
0    0.0044469754    0.051051278           0.0  0.0065815235  0.0008893951   
1     0.014324457            0.0     0.8811065           0.0           0.0   
2    0.0010765599   0.0010765599           0.0  0.0021531198  0.0026913998   
3    0.0024333484            0.0           0.0           0.0           0.0   
4             0.0            0.0  0.0010035756           0.0           0.0   
..            ...            ...           ...           ...           ...   
87   0.0040945313            0.0           0.0  0.0002559082           0.0   
88            0.0            0.0           0.0           0.0           0.0   
89            0.0            0.0           0.0           0.0           0.0   
90      0.7083379            0.0           0.0           0.0           0.0   
91  0.00049033243  0.00049033243    0.47660312           0.0  0.0009806649   

            5              6             7             8              9     \
0            0.0      0.0254367           0.0           0.0    0.043046724   
1            0.0            0.0           0.0           0.0            0.0   
2            0.0    0.006459359   0.010227319  0.0069976393   0.0037679595   
3            0.0   0.0030416856           0.0  0.0036500227    0.010950068   
4            0.0            0.0           0.0           0.0            0.0   
..           ...            ...           ...           ...            ...   
87           0.0   0.0002559082           0.0           0.0   0.0012795411   
88  0.0013365867            0.0           0.0  0.0022276444            0.0   
89           0.0  0.00067682005           0.0           0.0  0.00067682005   
90           0.0            0.0           0.0           0.0            0.0   
91           0.0  0.00049033243  0.0063743214  0.0024516622    0.004412992   

    ... 1430           1431 1432 1433           1434 1435 1436           1437  \
0   ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   
1   ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   
2   ...  0.0   0.0010678926  0.0  0.0   0.0008009195  0.0  0.0   0.0024027585   
3   ...  0.0  0.00037591136  0.0  0.0            0.0  0.0  0.0  0.00018795568   
4   ...  0.0  0.00019538656  0.0  0.0  0.00019538656  0.0  0.0            0.0   
..  ...  ...            ...  ...  ...            ...  ...  ...            ...   
87  ...  0.0  0.00062890793  0.0  0.0  0.00015722698  0.0  0.0   0.0022011776   
88  ...  0.0  0.00051748834  0.0  0.0  0.00051748834  0.0  0.0  0.00034499224   
89  ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   
90  ...  0.0            0.0  0.0  0.0  0.00024025064  0.0  0.0  0.00048050127   
91  ...  0.0            0.0  0.0  0.0            0.0  0.0  0.0            0.0   

   1438 1439  
0   0.0  0.0  
1   0.0  0.0  
2   0.0  0.0  
3   0.0  0.0  
4   0.0  0.0  
..  ...  ...  
87  0.0  0.0  
88  0.0  0.0  
89  0.0  0.0  
90  0.0  0.0  
91  0.0  0.0  

[92 rows x 1440 columns]

In [26]:
y_train = df[1]
y_train #training dataframe of outcomes as per the given features

0     0
1     0
2     2
3     2
4     2
     ..
87    1
88    2
89    2
90    2
91    2
Name: 1, Length: 92, dtype: int64

In [28]:
#using knn classifier to classify object
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors=3)   
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [49]:
while(5):
    print("enter the file name of image:\n")
    img = cv2.imread(input()) #test image
    
    #making test datframe
    dfs = []
    features = cd.describe(img)
    features = [str(f) for f in features]  
    feat = (",".join(features))
    dfs.append(feat)
    test = pd.DataFrame(dfs)
    x_test = test[0].str.split(",",expand = True)
    y_test = knn.predict(x_test)
    
    #displaying results
    if y_test == [0]:
        print('face')
    if y_test == [1]:
        print('night')
    if y_test == [2]:
        print('landscape')
        
    #displaying accuracy
    print(knn.score(x_test, y_test))
    
    #asking for next image
    print("do you want another classification:  yes/no")
    y = input()
    if y == "yes":
        continue  #again new image
    else:
        break #break the loop

enter the file name of image:

/home/subarna/Documents/ml_notebooks/nigh.jpeg
night
1.0
do you want another classification:  yes/no
yes
enter the file name of image:

/home/subarna/Documents/ml_notebooks/images.jpeg
face
1.0
do you want another classification:  yes/no
no
